In [2]:
!pip install pymongo
!pip install scikit-learn
!pip install nltk
import nltk
#nltk.download('stopwords')
from pymongo import MongoClient
import pymongo

In [9]:
import json
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import numpy as np

f = open('/Users/mazinrafi/Downloads/corona-out-3', 'r')

tweets = {}
retweets = {}
replies ={}
texts = {}
users = {}
hashtags = {}

# The specific keys to extract for tweets and users
tweet_keys = ['created_at', 'id_str', 'reply_count', 'retweet_count']
user_keys = ['name', 'screen_name', 'verified', 'followers_count', 'friends_count', 'created_at']

# Iterating over the input file
for line in f:
    if line == '\n':
        continue
    lx = json.loads(line)
    
    # Main extraction only for non-retweets
    if not 'retweeted_status' in lx or len(lx['retweeted_status']) == 0:
        li = dict((k, lx[k]) for k in tweet_keys if k in lx)
        # Store user ID and user screen name
        li['user'] = lx['user']['id']
        li['user_name'] = lx['user']['screen_name']
        
        # In some cases, text is truncated - here, extended_tweet full_text will contain the full version
        # If available, we use that, else the main text field is not truncated so we can use it
        if 'extended_tweet' in lx:    
            thisText = lx['extended_tweet']['full_text']
        else:
            thisText = lx['text']
        
        # Store the text (from whichever source) in both the tweet and the data source for keyword extraction
        li['text'] = thisText
        texts[lx['id']] = thisText
        
        # Identify hashtags using regex
        # The hashtags under entities are built from the main text field, which may have been truncated
        # So that subfield may be inaccurate
        hashtag_set = list(set(part[1:] for part in thisText.split() if part.startswith('#')))
        li['hashtags'] = hashtag_set
        li['hashtag_count'] = len(hashtag_set)
        
        # Track tweet IDs by hashtag, for later mapping
        for hashtag in hashtag_set:
            if hashtag in hashtags.keys():
                hashtags[hashtag].append(lx['id'])
            else:
                hashtags[hashtag] = [lx['id']]
        
        # Store this set of tweet details under this tweet ID
        li['_id'] = lx['id']
        
        tweets[lx['id']] = li
        
        # Track replies against their original tweets
        if 'in_reply_to_status_id' in lx:
            if not lx['in_reply_to_status_id'] in replies.keys():
                replies[lx['in_reply_to_status_id']] = 1
            else:
                replies[lx['in_reply_to_status_id']] = replies[lx['in_reply_to_status_id']] + 1
    else:
        # If a retweet, track against the original tweet
        orig_id = lx['retweeted_status']['id']
        
        if not orig_id in retweets.keys():
            retweets[orig_id] = 1
        else:
            retweets[orig_id] = retweets[orig_id] + 1
    
    # If this user has not been seen before, add their details into the user list
    if not lx['user']['id'] in users.keys():
        users[lx['user']['id']] = dict((k, lx['user'][k]) for k in user_keys if k in lx['user'])

f.close()

# Map retweets and replies to their original tweets, with counts
for rt in retweets.keys():
    if rt in tweets.keys():
        tweets[rt]['retweet_count'] = retweets[rt]

for rt in replies.keys():
    if rt in tweets.keys():
        tweets[rt]['reply_count'] = replies[rt]


# Extract unigrams, removing stopwords, for all keywords that appear at least 50 times
vectorizer = CountVectorizer(token_pattern=r"(?u)\b[a-z][a-z]+\b", stop_words=stopwords.words('english'), min_df = 50)

X = vectorizer.fit_transform(texts.values())

# The list of IDs and keywords used in the vectorizer
key_list = [*texts.keys()]
word_list = [*vectorizer.get_feature_names_out()]

# Track the number of keywords mapped to each ID, and store the count with the tweet details
x1 = X.toarray()

x2 = dict((key_list[w], len([word_list[t] for t in np.where(x1[w]>0)[0]])) for w in range(len(x1)))

for key_id in x2.keys():
    if key_id in tweets.keys():
        tweets[key_id]['keyword_count'] = x2[key_id]

# Track the IDs associated with each keyword, and store the mapping
X = np.transpose(X)

x1 = X.toarray()

x2 = dict((word_list[w], [key_list[t] for t in np.where(x1[w]>0)[0]]) for w in range(len(x1)))


with open("/Users/mazinrafi/Downloads/hashtag_tdm.json", "w") as outfile:
    json.dump(hashtags, outfile, indent=2)

    
keywords = dict((k, {'_id': k, 'tweets': v}) for k, v in x2.items())

hashtags = dict((k, {'_id': k, 'tweets': v}) for k, v in hashtags.items())


# upload tweets, keywords to mongodb, users to mysql

# s1 = [sum(i>0) for i in x1]

# s1 = [np.log(i) for i in s1]

# from matplotlib import pyplot as plt

# fig, ax = plt.subplots(figsize =(10, 7))
# ax.hist(s1)


In [10]:
client = MongoClient("mongodb://localhost:27017/")

db = client['TwitterDB']
collection = db['tweets']

# Insert tweets into collection
#set the id of the tweet to the id_str
for tweet_id, tweet_data in tweets.items():
    collection.insert_one(tweet_data)
    


In [62]:
len(tweets)

40804

In [14]:
client = MongoClient("mongodb://localhost:27017/")

# Create a new database
db = client["TwitterDB"]

# Create a new collection
collection = db["keywords"]


# Insert the data into the collection
for word_id, word_data in keywords.items():
    collection.insert_one(word_data)


In [17]:
len(keywords)

1453

In [15]:
#Upload the Hashtag json onto the server

client = MongoClient("mongodb://localhost:27017/")

# Create a new database
db = client["TwitterDB"]

# Create a new collection
collection = db["hashtags"]

# Insert the data into the collection
for word_id, word_data in hashtags.items():
    collection.insert_one(word_data)


In [18]:
len(hashtags)

11334

In [32]:
##Inserting the Users into MySQL 

!pip install mysql-connector-python
!pip install pytz
from mysql.connector import (connection)
import json
import mysql.connector

In [43]:
#You can download MySQL here: https://dev.mysql.com/downloads/installer/
#Connect to SQL database "localhost" after installing it.
#Note that the user and password must be defined during installation and re-used here.
#the user is "root" and the password is "gXt,:RsU#U-ws:3"
import mysql.connector

try:
    connection1 = mysql.connector.connect(host="localhost", user="root", password="gXt,:RsU#U-ws:3")
    print("Connected to MySQL server.")
except mysql.connector.Error as error:
    print("Failed to connect to MySQL server: {}".format(error))


Connected to MySQL server.


In [44]:
# create a new database. This database is called "new_database"

connection1 = mysql.connector.connect(host="localhost", user="root", password="gXt,:RsU#U-ws:3")
cursor = connection1.cursor()
cursor.execute("DROP DATABASE IF EXISTS new_database")
cursor.execute("CREATE DATABASE new_database")
print("Database created successfully!")


Database created successfully!


In [45]:
#create a new table. This one is called users_table and deletes any pre-existing ones. 
from datetime import datetime

connection1 = mysql.connector.connect(host="localhost", user="root", password="gXt,:RsU#U-ws:3",database="new_database")
cursor = connection1.cursor()

cursor.execute("DROP TABLE IF EXISTS users_table")
cursor.execute("""CREATE TABLE users_table (
    id BIGINT PRIMARY KEY,
    name VARCHAR(255),
    screen_name VARCHAR(255),
    followers_count INT,
    verified BOOLEAN,
    created_at DATETIME
)""")

print("Table created successfully!")

Table created successfully!


In [49]:
from datetime import datetime, timezone
#Insert the users into the Table
connection1 = mysql.connector.connect(
    host="localhost",
    user="root",
    password="gXt,:RsU#U-ws:3",
    database="new_database"
)
cursor = connection1.cursor()

used = set()


for user_id in users:
    if user_id in used:
        continue
    user_data = users[user_id]
    if 'created_at' not in user_data:
        continue
    created_at_str = user_data['created_at'][:-11] + user_data['created_at'][-5:] #(-5 extracts year, -11 extracts time)
    created_at = datetime.strptime(created_at_str, '%a %b %d %H:%M:%S %Y') #a is weekday, b is month, d is day of month, HMS, and Year
    created_at = created_at.replace(tzinfo=timezone.utc) #timezone imported from datetime module
    created_at_str = created_at.strftime('%Y-%m-%d %H:%M:%S') #converts to "YYYY-MM-DD HH:MM:SS" format
    sql = "INSERT INTO users_table (id, name, screen_name, followers_count, verified, created_at) VALUES (%s, %s, %s, %s, %s, %s)"
    values = (user_id, user_data['name'], user_data['screen_name'], user_data['followers_count'], user_data['verified'], created_at_str)
    cursor.execute(sql, values)
    connection1.commit()
    used.add(user_id)
    
print("Users inserted successfully!")


Users inserted successfully!


In [35]:
#Present some of the users as in a dataframe
import pandas as pd

connection1 = mysql.connector.connect(host="localhost", user="root", password="gXt,:RsU#U-ws:3",database="new_database")

cursor = connection1.cursor()

cursor.execute("USE new_database")

cursor.execute("""
    SELECT id, name, screen_name, followers_count 
    FROM users_table 
    LIMIT 5
    """)

result = cursor.fetchall()

df = pd.DataFrame(result, columns=['id', 'name', 'screen_name', 'followers_count'])

df

,id,name,screen_name,followers_count
0,767,Xeni Jardin 😷🏠,xeni,152608
1,5039,Jrome,jrome,642
2,5242,Ahmed Al Omran,ahmed,149844
3,11364,Kirti,Kits,922
4,12094,Nicholai Reinseth,Nicholai,277


In [52]:
#Prints top 10 users based on followers_count

connection1 = mysql.connector.connect(
    host="localhost",
    user="root",
    password="gXt,:RsU#U-ws:3",
    database="new_database"
)
cursor = connection1.cursor()
cursor.execute("USE new_database")

cursor.execute("""
    SELECT id, name, screen_name, followers_count 
    FROM users_table 
    ORDER BY followers_count DESC 
    LIMIT 10
""")

result = cursor.fetchall()

df = pd.DataFrame(result, columns=['id', 'name', 'screen_name', 'followers_count'])

df

,id,name,screen_name,followers_count
0,69183155,detikcom,detikcom,15927642
1,62513246,J.K. Rowling,jk_rowling,14608046
2,42606652,AajTak,aajtak,9704885
3,39240673,ABP News,ABPNews,9562582
4,240649814,TIMES NOW,TimesNow,9499328
5,56304605,Rajdeep Sardesai,sardesairajdeep,8947342
6,24744541,Le Monde,lemondefr,8808784
7,55507370,tvOneNews,tvOneNews,8787649
8,23343960,Kompas.com,kompascom,7678178
9,15016209,NTV,ntv,7429223


In [37]:
#print number of users (likely to be incorrect. Need to verify. Number is 80943): 
connection1 = mysql.connector.connect(host="localhost", user="root", password="gXt,:RsU#U-ws:3",database="new_database")

cursor = connection1.cursor()
cursor.execute("USE new_database")
cursor.execute("SELECT COUNT(*) FROM users_table")
result = cursor.fetchone()
print("Number of users in the table:", result[0])

cursor.close()
connection1.close()

Number of users in the table: 80943


In [38]:
#Directly from file

import json
with open('/Users/mazinrafi/Downloads/corona-out-3.json') as f:
    file_data = json.load(f)

unique_users = set()

for i in file_data:
    unique_users.add(i['user']['id'])

num_unique_users = len(unique_users)
print("Number of unique users:", num_unique_users)

Number of unique users: 80943


In [19]:
len(users)

80943

In [ ]:
#For testing purposes if needed. Do not run the cell

#Drop DB. May need to adjust the connection accordingly. 
connection1 = mysql.connector.connect(host="localhost", user="root", password="gXt,:RsU#U-ws:3",database="new_database")
cursor = connection1.cursor()
sql= "DROP DATABASE my_database"
cursor.execute(sql)


In [58]:

#Drop Table
connection1 = mysql.connector.connect(host="localhost", user="root", password="gXt,:RsU#U-ws:3",database="new_database")
cursor = connection1.cursor()
sql= "DROP TABLE users"
cursor.execute(sql)

#Test to see if your connection works: 

import mysql.connector
import json

try:
    connection1 = mysql.connector.connect(host="localhost", user="root", password="gXt,:RsU#U-ws:3",database="corona_out_2_db")
    print("Connected to MySQL server and database.")
except mysql.connector.Error as error:
    print("Failed to connect to MySQL server: {}".format(error))

In [109]:
#Side Notes

#need to keep the Python
#JSON Import Utility
#Primary Key needed
#Database needed and ID
# create a new database. This database is called "new_database"
